# Recommending Children's Books
### Technical Report

#### Problem Statement:
Finding new books for children to read that they will enjoy and which are at an appropriate reading level is a challenge faced by educators and parents alike. I would like to help address this problem by building a recommender system based upon the reading level and subject matter of a book that the child has already read and liked.

I pulled my data from [Project Gutenberg](http://www.gutenberg.org/wiki/Main_Page), which is a site that offers over 53,000 free ebooks. This ended up limiting the useage of my recommender to just books within Project Gutenberg. All of the books that are offered are free because their copyright has expired in the United States, which means that the I cannot offer recommendations for modern books. However, my pipeline would work with modern books if they did become available to me.

### Contents
1. Introduction and background
2. Obtaining the books and metadata from Project Gutenberg
3. Parsing through the text of the books with textstat, count vectorizer, and tf-idf vectorizer
4. Attempting to predict Childrens Books
5. The Engine and how it works
4. Results and Anaysis

### 1. Introduction and Background

In order to properly understand and enjoy a text, and thus continue to develop as a reader, it is important for children to have access to books that are both interesting and engaging for them and within their [Zone of Proximal Development](https://www.soe.vt.edu/tandl/pdf/Mesmer/Presentation_Mesmer_VSRA_2008_MatchingKids.pdf). 

The Zone of Proximal Development, usually abbreviated as ZPD, is a concept describes what a child can do on their own, what they can do with help, and what is beyond their ability. In reading development, we want to ensure that children are both accurately reading the words and that they are comprehending what they are reading, or else they will become frustrated with the text. 


#### Reading Levels

A big part of my recommendation engine is based upon reading level, of which there are many different measures. I wanted to ensure that the texts being recommended were within the ZPD of the reader so I needed to find a way to quantify the gap between what the reader can and cannot read. Some reading levels work with formulas based upon word length and sentence length and others also take into account word difficulty and lists of grade level words. I have chosen to use [Flesch Kincaid Grade Level](https://en.wikipedia.org/wiki/Flesch%E2%80%93Kincaid_readability_tests) for the measure of reading level metric in this project. I made this choice because it is a widely accepted measure that is also directly calculable without importing lists of grade level words. I was wary of using lists of grade level words considering that most of the books in my dataset were written well before the current standards of grade level words and I want to be able to compare books accross different time periods. 

Flesch Kincaid grade level is calculated using the following formula: 

$$0.39\left(\frac{\text{total words}}{\text{total sentences}}\right)+11.8\left(\frac{\text{total syllables}}{\text{total words}}\right)-15.59$$

The resulting number corresponds somewhat to what grade level the child is in in school, though in reality students can vary widely in reading level within each grade.

There is another realated measure called the Flesch reading-ease test, in which higher scores indicate material that is easier to read; lower numbers mark passages that are more difficult to read. Scores above 90 indicate that the text is very easy to read and can be easily understood by an average 11-year-old student. Scores below 30 indicate that the text is very difficult to read and best understood by university graduates. The formula for Flesch reading ease is:

$$206.835-1.015\left(\frac{\text{total words}}{\text{total sentences}}\right)-84.6\left(\frac{\text{total syllables}}{\text{total words}}\right)$$

In the end I chose to use the Flesch Kincaid grade level metric as it was more intuitive to have the reading level go up as the difficulty level of the book went up. I did calculate both of these reading levels for each book. From this point forward when I refer to reading level I will mean Flesch Kincaid grade level. 

#### The Recommender System

The basic premise of my recommender system was to take as input the title of a book that the reader recently enjoyed reading and to output a list of recommended books. I did this by first obtaining a subset the library of available books to only include books that were within the ZPD of the reader. This was determined by finding the reading level of the book that the reader enjoyed and returning books that were within an acceptable range of reading levels close to the book they entered in. I determined this to be between one grade level below and one quarter of a grade level above the book they entered in, after reading research about reading level and ZPD. Depending on the book, this usually narrowed down the books available to between one and three thousand, sometimes more.

Once I had a subset of acceptable books, I determined how similar the subject matter of the books were using the Jaccard Similarity of the lists of subjects that were determined by the person who submitted the text to Project Gutenberg. However, I also wanted to compare the actual text of the books to each other, so I found the cosine distances between the count vectorized texts of the books to each other as well as the cosine distances of the tf-idf vectorized texts of the books to each other. This gave me three different measures of text similarity, which I will compare in the analysis section below.


### 2. Obtaining and cleaning the book metadata and the text of the books from Project Gutenberg

The code I used to make a dataframe of the metadata about the books, some of it credited to [Andreas van Cranenburgh](https://gist.github.com/andreasvc/b3b4189120d84dec8857), can be found [here](https://github.com/lprowell/gutenberg-book-recommender/blob/master/getting_meta_df.ipynb).

The code I used to pull the text of the books down from the Gutenberg website can be found [here](https://github.com/lprowell/gutenberg-book-recommender/blob/master/getting_book_text_cvec_subjects.ipynb)

#### Metadata
My first challenge was to get the data about the books that I had available to me. I needed to get information about the title, author, and subjects of the books, as well as the type of file, the language it was written in, and the library of congress reference code for the book.

Once I had the data in a readable format, I needed to narrow down my dataframe to only include what I was able to use for this project. Here are the steps that I took to clean up my dataframe:
- Removed row 0 (there is no book with id 0)
- Removed anything that is not in a text format, such as audiobooks or datasets.
  - *Interestingly, the 'Dataset' type includes data like the first 1 million digits of 1/pi and some human chromosomes. I would love to come back and so some analysis on these datasets in a future project!*
- Removed books that were not exclusively written in English
  - *There were several books which were written in multiple languages, and I found that to be quite interesting but again, an endeavor to take on in a future project.*
- Removed books to which access has been blocked (there are several books like this, probably for copyright reasons)

When I first loaded in the dataframe, I had 54,124 rows. After going through the narrowing down process described above, I was left with 41,429 books, all text, all written in English, all with titles and authors. My current columns were: 

Column Name | Description
---|---
'id' | A unique number for each book, which can be used to find the book on the Gutenberg website.  
'title' | The title of the book 
'author' | The author of the book 
'LCC' | The [library of congress classification](https://en.wikipedia.org/wiki/Library_of_Congress_Classification) of the book. A book could have more than one classification.
'downloads' | The number of times the book has been downloaded from the Gutenberg website.
'subjects' | The subjects of the book, which were designated by the person who submitted the book to the Gutenberg website
'formats' | The formats that the book is available in. I was interested in the plain text format for this project, but there were also kindle versions available. 
'authoryearofbirth' | The year the author was born, if available 
'authoryearofdeath' | The year the author died, if available
'type' | The options here were Text, Dataset, StillImage, MovingImage, Sound, Image, or Collection. I was only interested in Text.
'language' | There were lots of languages available, and some books that were written in multiple languages. I was only interested in books written in English for the purposes of this project.

One column that I could not get from Gutenberg but I wish I had was the year the book was published. If I were to expand upon this project I would try and get this data from another source and merge it with what I have here, but for the purposes of my current project I determined that this was unnecessary.

#### Starting to look at subjects

In order to begin sorting the books in some way, I wanted to start looking at the subjects listed for the books. I decided to change the format they were in to make this a little bit easier.

The subjects were were originally in the form of a set of between one and twenty-five subjects with each subject having a few parts connected by '--'. For example, Moby Dick had the subject:
'{Ship captains -- Fiction, Whaling ships -- Fiction, Psychological fiction, Whaling -- Fiction, Whales -- Fiction, Ahab, Captain (Fictitious character) -- Fiction, Sea stories, Adventure stories, Mentally ill -- Fiction}'.


I went through each item in the set and split it on the '--' to make a list of all the subjects listed. I also made them lowercase. Moby Dick's subjects ended up looking like this:
['adventure stories', 'ahab', 'captain fictitious character', 'fiction', 'mentally ill', 'psychological fiction', 'sea stories', 'ship captains', 'whales', 'whaling', 'whaling ships']
This would make it easier to find books with similar subjects later on.

#### Book Text

Before I retreived the text of the books, I decided to narrow down my dataframe some more to make the volume of text I was retrieving more manageable. I used the subjects column to retrieve books that were either fiction or literature. I made this choice to exclude books that I would not recommend, such as the US Consititution, from my dataframe.

After removing all books that were neither literature or fiction, I was left with 16,049 books, which was much more manageable. A challenge that I had while creating a dataframe of the book text was that it became quite large rather quickly. To help mitigate this, I created 23 files with 700 books in each instead of one enormous file.  

My book dataframes were simply one column with the unique book_id of the book and one with the text of the book. This would allow me to match up the book with the metadata later.

### 3. Parsing through the text of the books with textstat, count vectorizer, and tf-idf vectorizer

Now that I had the text of the books, I could begin to parse through it to find out things like the character count, word count, syllable count, amount of dialog, reading level, word usage and frequency, and term frequency–inverse document frequency of words.

#### Text Stats
The code I used to extract the stats from the text can be found [here](https://github.com/lprowell/gutenberg-book-recommender/blob/master/book_stats.ipynb).

I started out by using a package called [textstat](https://pypi.python.org/pypi/textstat/0.1.4) along with a couple of functions that I wrote to create the following new columns:

Column | Description
---|----
character count | The total number of characters in the book (textstat)
word count | The total number of words in the book (textstat lexicon count)
sentence count | The total number of sentences in the book (textstat)
syllable count | The total number of syllables in the book (textstat)
average sentence length | $\frac{\text{word count}}{\text{sentence count}}$, calculated directly
avgerage syllables per word |  $\frac{\text{syllable count}}{\text{word count}}$, calculated directly
dialogue | The number of quotation marks in the text divided by 2
dialogue per sentence | $\frac{\text{dialogue}}{\text{sentence count}}$, calculated directly
flesch reading ease | A measure of reading level, described in the introduction above.
flesch kincain grade | Another, related, measure of reading level, also described in the introduction.

This allowed me to sort out the texts by reading levels. The easiest book in my entire library turned out to be a reading primer called [Little Present](http://www.gutenberg.org/files/24703/24703.txt) by an unknown author with reading level 2.39545 and the most difficult was [The King of Pirates](http://www.gutenberg.org/files/37992/37992.txt) by Daniel Defoe with a reading level of 30.2999.

#### Count Vectorizer

The code I used to count vectorize the texts can be found [here](https://github.com/lprowell/gutenberg-book-recommender/blob/master/getting_book_text_cvec_subjects.ipynb).

My next step in investigating the text in the books was to vectorize it using [sklearn's count vectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). This allowed me to see what words were most commonly used in individual books and then to compare the books to each other using [cosine distances](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_distances.html) of the resulting vectors. The only parameter I put into my count vectorizor was stopwords = 'english', which means that my vectorized texts did not count words that are used very commonly in english (such as 'the', 'if', etc.). I chose to use this list of stopwords so that I could highlight actual differences between the texts instead of seeing distances that were watered down due to the very commonly used words being in all of the books.

#### TF-IDF Vectorizer

The code I used to tf-idf vectorize the texts can be found [here](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

I wanted to compare the recommendations I got using distances of count vectorized texts with another measure so that I could analyze how the quality of my recommendations compared. I chose to use the term frequency–inverse document frequency ([tf-idf](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)) vectorizer for that. After trying out many different combinations of parameters I settled on the ones shown below, which 
gave me the best distribution of terms. The goal was to find the right parameters so that I could clearly see similarities between texts without the noise caused by words that are extremely frequent in all texts or words that rarely show up witin a text.

tvec = TfidfVectorizer(strip_accents='unicode',
    decode_error='replace',
    lowercase=True,
    stop_words='english',
    min_df = 0.1, 
    max_df=0.90, 
    max_features = 5000)
    


<img src="../gutenberg-book-recommender/plots_pics/tfidf_hists.png" width="600" height="800" />



### 4. Attempting to predict Children's Books

To see the code for my logistic regression to attempt to predict Children's books, go [here](https://github.com/lprowell/gutenberg-book-recommender/blob/master/predicting_children.ipynb).

The next thing I attempted was unsuccessful, but the general goal was to use the reading level statistics columns from the previous step and try to predict if a book was written for children or not. I made a column called "childrens" to be my target and then I tried a gridsearch to find the best parameters. After several tries for the logistic regression with many different combinations of predictors, I realized that the problem was not so much my predictors but my target. I had no reliable way to label whether a book was written for children or adults, so my training data was flawed. 

However, I do not think that the failure of the metric of reading level that I used to predict children's books well reflects poorly on the quality of my recommender system, but rather on the labelling of the books. It doesn't really matter, in the end, if a book is filed in the children's, young adult's or adult's section as long as the book is at an appropriate reading level and is engaging for the reader.

### 5. The Engine and How it Works

At the moment I have two files that each have my recommendation system in them. The [long version]() includes the code for the system on lots of seperate lines along with some EDA as I go. The [slightly more concise version]() just has a function that takes a book title as input and outputs a list of recommended books.

The flow of my pipeline is as follows:
- load in any packages I need as well as the dataframes
- the user enters the title of a book they liked
- the metadata frame is subset to only 